# Create Marking Form
1. Convert PDF into images.
2. Highlight student's answer

Install Linux tools and only required for the first run.

In [1]:
!sudo apt-get update
!sudo apt-get -y install poppler-utils

Get:1 http://deb.debian.org/debian bookworm InRelease [151 kB]
Get:2 http://deb.debian.org/debian bookworm-updates InRelease [55.4 kB]       
Get:3 http://deb.debian.org/debian-security bookworm-security InRelease [48.0 kB]
Get:4 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]
Get:5 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]
Get:6 http://deb.debian.org/debian bookworm/main amd64 Packages [8789 kB]
Get:7 http://deb.debian.org/debian bookworm-updates/main amd64 Packages [8856 B]
Get:8 http://deb.debian.org/debian-security bookworm-security/main amd64 Packages [216 kB]
Get:9 https://dl.yarnpkg.com/debian stable/main amd64 Packages [10.9 kB]
Get:10 https://dl.yarnpkg.com/debian stable/main all Packages [10.9 kB]
Get:11 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3446 kB]
Get:12 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1588 kB]
Fetched 14.3 MB in 1s (14.1 MB/s)                          
Reading package lists

## Define the student script

In [2]:
pdf_file = "../data/TestScript.pdf"

In [3]:
# extrat file name from pdf_file
import os

file_name = os.path.basename(pdf_file)
file_name = os.path.splitext(file_name)[0]
base_path = "../marking_form/" + file_name
base_path_images = base_path + "/images/"
base_path_annotations = base_path+"/annotations/"
# create directory tree for base_path_images
os.makedirs(base_path_images, exist_ok=True)
os.makedirs(base_path_annotations, exist_ok=True)

## Convert PDF to JPG

In [4]:
# read pdf and convert to images
# https://stackoverflow.com/questions/46184239/how-to-convert-pdf-to-image-using-python
from pdf2image import convert_from_path
import os

pages = convert_from_path(pdf_file, fmt='jpeg')
# extrat file name from pdf_file
file_name = os.path.basename(pdf_file)
file_name = os.path.splitext(file_name)[0]

for count, page in enumerate(pages):
    page.save(f'{base_path_images}{count}.jpg', 'JPEG')

In [5]:
import base64
import json

def update_json_file(annotations, path):
    with open(path, "w") as f:
        json.dump(annotations, f, indent=4)   

def image_to_data_url(filename):
    ext = filename.split(".")[-1]
    prefix = f"data:image/{ext};base64,"
    with open(filename, "rb") as f:
        img = f.read()
    return prefix + base64.b64encode(img).decode("utf-8")

In [6]:
import boto3
import json

client = boto3.client(service_name="bedrock-runtime")

messages = [
    {"role": "user", "content": [{"text": "Write a short poem"}]},
]

model_response = client.converse(
    modelId="us.amazon.nova-lite-v1:0", 
    messages=messages
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print("\n[Response Content Text]")
print(model_response["output"]["message"]["content"][0]["text"])


[Full Response]
{
  "ResponseMetadata": {
    "RequestId": "1ccc183e-a685-485a-84bb-c7f5350504e5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 11 Dec 2024 09:06:28 GMT",
      "content-type": "application/json",
      "content-length": "466",
      "connection": "keep-alive",
      "x-amzn-requestid": "1ccc183e-a685-485a-84bb-c7f5350504e5"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "In twilight's gentle, soft embrace,\nA whisper of the evening's grace,\nStars emerge, a silent gleam,\nAs night unveils its quiet dream.\n\nThe moon, a silver, watchful eye,\nReflects on waves that gently sigh,\nA dance of light, a fleeting scene,\nIn nature's tranquil, timeless sheen."
        }
      ]
    }
  },
  "stopReason": "end_turn",
  "usage": {
    "inputTokens": 4,
    "outputTokens": 68,
    "totalTokens": 72
  },
  "metrics": {
    "latencyMs": 696
  }
}

[Response Content 

In [36]:
import boto3
import json

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
)

MODEL_ID = "us.amazon.nova-pro-v1:0"


def ocr(prompt:str, filePath:str):       
    with open(filePath, "rb") as f:
        binary_data = f.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode("utf-8")

    # Define your system prompt(s).
    system_list = [    {
            "text": "You are a assistent for object annotation."
        }
    ]
    # Define a "user" message including both the image and a text prompt.
    message_list = [
        {
            "role": "user",
            "content": [
                {
                    "image": {
                        "format": "jpeg",
                        "source": {"bytes": base64_string},
                    }
                },
                {
                    "text": prompt
                }
            ],
        }
    ]
    # Configure the inference parameters.
    inf_params = {"max_new_tokens": 5120, "top_p": 0.1, "top_k": 20, "temperature": 0.3}

    native_request = {
        "schemaVersion": "messages-v1",
        "messages": message_list,
        "system": system_list,
        "inferenceConfig": inf_params,
    }
    # Invoke the model and extract the response body.
    response = client.invoke_model(modelId=MODEL_ID, body=json.dumps(native_request))
    model_response = json.loads(response["body"].read())
    # Pretty print the response JSON.
    print("[Full Response]")
    print(json.dumps(model_response, indent=2))
    # Print the text content for easy readability.
    content_text = model_response["output"]["message"]["content"][0]["text"]
    print("\n[Response Content Text]")
    print(content_text)
    return content_text

In [37]:
import json

prompt = """Extract the coordinate of restangle cells bound box from the image.
Label is the question number such as Q1, Q2, Q3, etc. and it is the text inside the top right of the cell.
Cells are not in the form of table, they are just rectangle boxes.
Bound box is no overlap with other bound box.
Output should be in the following JSON Array format:
[
    {
        "x": 152,
        "y": 313,
        "width": 702,
        "height": 243,
        "label": "1"
    },
    {
        "x": 152,
        "y": 313,
        "width": 702,
        "height": 243,
        "label": "2"
    },
    .....
]
Do not include any explanations, only provide a RFC8259 compliant JSON response following this format without deviation in the following format.
Output:

"""

aiAnnoation = {}
number_of_pages = 1
for i in range(number_of_pages):
    image_path = base_path_images + f"{i}.jpg"
    result = ocr(prompt, image_path) 
    result = result.strip("```json\n").strip("```")
    result = json.loads(result)
    aiAnnoation[str(i)] = result

print(aiAnnoation)

[Full Response]
{
  "output": {
    "message": {
      "content": [
        {
          "text": "[\n    {\n        \"x\": 51,\n        \"y\": 100,\n        \"width\": 240,\n        \"height\": 30,\n        \"label\": \"Q1\"\n    },\n    {\n        \"x\": 300,\n        \"y\": 100,\n        \"width\": 240,\n        \"height\": 30,\n        \"label\": \"Q2\"\n    },\n    {\n        \"x\": 549,\n        \"y\": 100,\n        \"width\": 391,\n        \"height\": 30,\n        \"label\": \"Q3\"\n    },\n    {\n        \"x\": 51,\n        \"y\": 139,\n        \"width\": 240,\n        \"height\": 30,\n        \"label\": \"Q4\"\n    },\n    {\n        \"x\": 300,\n        \"y\": 139,\n        \"width\": 240,\n        \"height\": 30,\n        \"label\": \"Q5\"\n    },\n    {\n        \"x\": 549,\n        \"y\": 139,\n        \"width\": 391,\n        \"height\": 30,\n        \"label\": \"Q6\"\n    },\n    {\n        \"x\": 51,\n        \"y\": 178,\n        \"width\": 240,\n        \"height\": 30,\n

In [38]:
import copy
backup = copy.deepcopy(aiAnnoation)

In [39]:
from PIL import Image

# Open an image file
with Image.open(image_path) as img:
    # Get width and height
    width, height = img.size

print(f"Width: {width}, Height: {height}")

Width: 1700, Height: 2200


In [50]:
import json
import copy
aiAnnoation = copy.deepcopy(backup)

x_scale = width / 1000.0
y_scale = height / 1000.0
# x_scale = 1
# y_scale = 1
for i in range(number_of_pages):
    for item in aiAnnoation[str(i)]:
        item['x'] = int(round(item['x'] * x_scale))
        item['y'] = int(round(item['y'] * y_scale)) 
        item['width'] = int(round(item['width'] * x_scale))
        item['height'] = int(round(item['height'] * y_scale))


ai_annotations_path = base_path_annotations + "ai_annotations.json"

# Save the aiAnnoation variable to a JSON file
with open(ai_annotations_path, "w") as f:
    json.dump(aiAnnoation, f)


In [51]:
from jupyter_bbox_widget import BBoxWidget
import ipywidgets as widgets
import json
import glob

page = 1
pageAndBoxingBoxes={}

files = sorted(glob.glob(base_path_images + "*.jpg"))

w_progress = widgets.IntProgress(value=0, max=len(files), description="Progress")
annotations_path = base_path_annotations + "annotations.json"
ai_annotations_path = base_path_annotations + "ai_annotations.json"

annotations = {}
# if annotations_path exists, load annotations from it
if os.path.exists(ai_annotations_path):
    with open(ai_annotations_path, "r") as f: 
        annotations = json.load(f) 

if os.path.exists(annotations_path):
    with open(annotations_path, "r") as f: 
        annotations = json.load(f) 

question_widget = widgets.Text(value="", placeholder="", description="Question:")

w_bbox = BBoxWidget(
    image=image_to_data_url(files[0])   
)
w_bbox.attach(question_widget, name="label")
w_bbox.bboxes = annotations[str(w_progress.value)] if str(w_progress.value) in annotations else []

# when Skip button is pressed we move on to the next file
def on_skip():
    w_progress.value += 1
    # open new image in the widget
    image_file = files[w_progress.value]
    w_bbox.image = image_to_data_url(image_file)     
    w_bbox.bboxes = annotations[str(w_progress.value)] if str(w_progress.value) in annotations else []


w_bbox.on_skip(on_skip)

# when Submit button is pressed we save current annotations
# and then move on to the next file
def on_submit():
    image_file = files[w_progress.value]
    # save annotations for current image
    annotations[str(w_progress.value)] = w_bbox.bboxes
    update_json_file(annotations, annotations_path)
    # move on to the next file
    on_skip()


w_bbox.on_submit(on_submit)
w_out = widgets.Output()

def on_bbox_change(change):
    w_out.clear_output(wait=True)
    with w_out:
        print(json.dumps(change["new"], indent=4))
        pageAndBoxingBoxes[w_progress.value] = change["new"]


w_bbox.observe(on_bbox_change, names=["bboxes"])

w_container = widgets.VBox(
    [
        widgets.HBox(
            [
                question_widget            
            ]
        ),
        w_progress,
        w_bbox,
        w_out,
    ]
)
w_container